# Building Applications with LLMs

- [Skool Link](https://www.skool.com/data-alchemy/classroom/a455582e?md=cffccd9c21af41e0bc3669610ce3bc39)
- [YouTube](https://www.youtube.com/watch?time_continue=1783&v=NYSWn1ipbgg&embeds_referring_euri=https%3A%2F%2Fwww.skool.com%2F&embeds_referring_origin=https%3A%2F%2Fwww.skool.com&source_ve_path=Mjg2NjY&feature=emb_logo)
- [LangChain Experiments](https://github.com/daveebbelaar/langchain-experiments)

In [1]:
import os
import dotenv

dotenv_path = dotenv.find_dotenv()
dotenv.load_dotenv(dotenv_path)

True

In [5]:
import os
import nest_asyncio
import pandas as pd
from dotenv import find_dotenv, load_dotenv
from langsmith import Client
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.smith import RunEvalConfig, run_on_dataset

# To Avoid the Error on Jupyter Notebook (RuntimeError: This Event Loop Is Already Running)
# Patch Asyncio To Allow Nested Event Loops

# nest_asyncio.apply()

In [4]:
!pip show langchain --version

Name: langchain
Version: 0.1.13
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [7]:
load_dotenv(find_dotenv())
os.environ["LANGCHAIN_API_KEY"] = str(os.getenv("LANGCHAIN_API_KEY"))
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "langsmith-tutorial"

In [8]:
# Load the LangSmith Client
client = Client()

# Test run
llm = ChatOpenAI()
llm.predict("Hello, world!")

/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mpaz/Library/Caches/pypoetry/virtualenvs/poetry-030-oHjfFdrQ-py3.11/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Hello! How can I assist you today?'

In [9]:
# 1. Create a Dataset (Only Inputs, No Output)

example_inputs = [
    "a rap battle between Atticus Finch and Cicero",
    "a rap battle between Barbie and Oppenheimer",
    "a Pythonic rap battle between two swallows: one European and one African",
    "a rap battle between Aubrey Plaza and Stephen Colbert",
]

dataset_name = "Rap Battle Dataset"

# Storing inputs in a dataset lets us
# run chains and LLMs over a shared set of examples.
dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Rap battle prompts.",
)

for input_prompt in example_inputs:
    # Each example must be unique and have inputs defined.
    # Outputs are optional
    client.create_example(
        inputs={"question": input_prompt},
        outputs=None,
        dataset_id=dataset.id,
    )

In [10]:
# 2. Evaluate Datasets with LLM

eval_config = RunEvalConfig(
    evaluators=[
        # You can specify an evaluator by name/enum.
        # In this case, the default criterion is "helpfulness"
        "criteria",
        # Or you can configure the evaluator
        RunEvalConfig.Criteria("harmfulness"),
        RunEvalConfig.Criteria("misogyny"),
        RunEvalConfig.Criteria(
            {
                "cliche": "Are the lyrics cliche? "
                "Respond Y if they are, N if they're entirely unique."
            }
        ),
    ]
)

run_on_dataset(
    client=client,
    dataset_name=dataset_name,
    llm_or_chain_factory=llm,
    evaluation=eval_config,
)

View the evaluation results for project 'cold-attitude-79' at:
https://smith.langchain.com/o/7ee11ef8-9b08-4ce8-9115-3b29dcdab1a3/datasets/608f4a25-2baf-4246-b255-36b311e26bb7/compare?selectedSessions=5b179a34-4105-4016-8017-d93c649529c8

View all tests for Dataset Rap Battle Dataset at:
https://smith.langchain.com/o/7ee11ef8-9b08-4ce8-9115-3b29dcdab1a3/datasets/608f4a25-2baf-4246-b255-36b311e26bb7
[------------------------------------------------->] 4/4

{'project_name': 'cold-attitude-79',
 'results': {'45c6b65a-6b5b-4a29-b25c-b35a5b3babe8': {'input': {'question': 'a rap battle between Aubrey Plaza and Stephen Colbert'},
   'feedback': [EvaluationResult(key='helpfulness', score=1, value='Y', comment='The criteria for this task is "helpfulness: Is the submission helpful, insightful, and appropriate?"\n\n1. Helpfulness: The submission is helpful in the sense that it provides a creative and entertaining response to the input. It imagines a rap battle between Aubrey Plaza and Stephen Colbert, which was the task given.\n\n2. Insightfulness: The submission is insightful as it uses characteristics of both Aubrey Plaza and Stephen Colbert to create their respective rap verses. It shows an understanding of their public personas.\n\n3. Appropriateness: The submission is appropriate as it sticks to the task given and does not include any offensive or inappropriate content.\n\nBased on these points, the submission meets all the criteria. \n\nY', 

In [11]:
# 1. Create a Dataset From a List of Examples (Key-Value Pairs)

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

dataset_name = "Elementary Animal Questions"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions and answers about animal phylogenetics.",
)

for input_prompt, output_answer in example_inputs:
    client.create_example(
        inputs={"question": input_prompt},
        outputs={"answer": output_answer},
        dataset_id=dataset.id,
    )

In [12]:
# 2. Create a Dataset From Existing Runs

dataset_name = "Example Dataset"

# Filter runs to add to the dataset
runs = client.list_runs(
    project_name="evaluators",
    execution_order=1,
    error=False,
)

dataset = client.create_dataset(dataset_name, description="An example dataset")

for run in runs:
    client.create_example(
        inputs=run.inputs,
        outputs=run.outputs,
        dataset_id=dataset.id,
    )

In [13]:
# 3. Create a Dataset From a Dataframe

# Create a Dataframe

example_inputs = [
    ("What is the largest mammal?", "The blue whale"),
    ("What do mammals and birds have in common?", "They are both warm-blooded"),
    ("What are reptiles known for?", "Having scales"),
    (
        "What's the main characteristic of amphibians?",
        "They live both in water and on land",
    ),
]

df_dataset = pd.DataFrame(example_inputs, columns=["Question", "Answer"])
df_dataset.head()

,Question,Answer
0,What is the largest mammal?,The blue whale
1,What do mammals and birds have in common?,They are both warm-blooded
2,What are reptiles known for?,Having scales
3,What's the main characteristic of amphibians?,They live both in water and on land
